In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

## Loading Iris

In [3]:
iris = load_iris()

In [4]:
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

## Converting data into X, y

In [5]:
X = iris['data']
y = iris['target']

## Standard Scaling of Features

In [6]:
scaler = StandardScaler()

In [7]:
X_scaled = scaler.fit_transform(X) # use to trnasform them into standar scaling

## Conversion of data into tensors

In [8]:
X_tensor = torch.tensor(X_scaled, dtype=torch.float32) # converting them into tensor format
y_tensor = torch.tensor(y, dtype=torch.long)

## Data Partitioning

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2)

In [10]:
train_ds = TensorDataset(X_train, y_train) # it is a requirement of the syntex for the model
test_ds = TensorDataset(X_test, y_test)

In [11]:
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=16) # batch size should not be very big nor very small i.e. 8, 16

In [12]:
class AnnModel(nn.Module):
    def __init__(self):
        super(AnnModel, self).__init__()
        # Define layers
        self.fc1 = nn.Linear(4, 16)
        self.Relu = nn.ReLU()
        self.fc2 = nn.Linear(16, 3)

    # defining the flow of the data
    def forward(self, x):
        out = self.fc1(x)
        out = self.Relu(out)
        out = self.fc2(out)
        return out

In [13]:
model = AnnModel()

In [14]:
citerion = nn.CrossEntropyLoss() # calculates the difference
optimizer = optim.Adam(model.parameters(), lr=0.01) # updates the weights and biases at given rate

In [15]:
epochs = 50 # one complete cycle through the entire training dataset during training

In [16]:
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad() # optimizer changes weight and bias at given rate
        outputs = model(batch_x) # this line applies model on batch_x (features) and store output in variable 'output' 
        loss = citerion(outputs, batch_y) # Forward Propogation
        loss.backward() # Backward Propogation
        optimizer.step() # changes the weights and bias according to learning
        # by this point the modle is completed and trained the next lines are for personal evaluation
        total_loss +=loss.item()
    print(f"Epoch: {epoch}, Loss: {total_loss}")

Epoch: 0, Loss: 8.768027305603027
Epoch: 1, Loss: 6.200192451477051
Epoch: 2, Loss: 4.435360133647919
Epoch: 3, Loss: 3.386374995112419
Epoch: 4, Loss: 2.6338627636432648
Epoch: 5, Loss: 2.3777518421411514
Epoch: 6, Loss: 1.9262157529592514
Epoch: 7, Loss: 1.6659545302391052
Epoch: 8, Loss: 1.5762218236923218
Epoch: 9, Loss: 1.4107603654265404
Epoch: 10, Loss: 1.1518041044473648
Epoch: 11, Loss: 1.047946721315384
Epoch: 12, Loss: 0.9335503093898296
Epoch: 13, Loss: 0.8531984426081181
Epoch: 14, Loss: 0.8575781546533108
Epoch: 15, Loss: 0.859555434435606
Epoch: 16, Loss: 0.6759940981864929
Epoch: 17, Loss: 0.6538369655609131
Epoch: 18, Loss: 0.6243088617920876
Epoch: 19, Loss: 0.6190301962196827
Epoch: 20, Loss: 0.6120967287570238
Epoch: 21, Loss: 0.5645208433270454
Epoch: 22, Loss: 0.5321470461785793
Epoch: 23, Loss: 0.5863674003630877
Epoch: 24, Loss: 0.5204787012189627
Epoch: 25, Loss: 0.5006831707432866
Epoch: 26, Loss: 0.4735823282971978
Epoch: 27, Loss: 0.4616673165000975
Epoch: 2

In [23]:
with torch.no_grad(): # this lines tells: don't change anything the model is already trained
    lst = []
    for batch_x, batch_y in test_loader:
        outputs = model(batch_x) # the maximum number of output is the most likely prediction
        _, prediction = torch.max(outputs, 1) # this function gives the position of maximum output i.e. 0, 1, 2
        print("Prediction:    ", prediction)
        print("Actual Results:", batch_y) # now we are comparing the model predictions with actual results
        lst.append(prediction) # a list containing all batches of predictions

Prediction:     tensor([1, 2, 1, 2, 1, 1, 0, 1, 1, 2, 0, 1, 2, 0, 1, 1])
Actual Results: tensor([1, 2, 1, 2, 2, 1, 0, 1, 1, 2, 0, 1, 2, 0, 1, 1])
Prediction:     tensor([0, 1, 2, 2, 0, 2, 1, 1, 2, 1, 1, 1, 0, 1])
Actual Results: tensor([0, 1, 2, 2, 0, 2, 1, 1, 2, 1, 1, 1, 0, 1])


In [24]:
lst

[tensor([1, 2, 1, 2, 1, 1, 0, 1, 1, 2, 0, 1, 2, 0, 1, 1]),
 tensor([0, 1, 2, 2, 0, 2, 1, 1, 2, 1, 1, 1, 0, 1])]

In [33]:
# now we are changing the predictions with actual names at given positions by manually looking at them
for i in lst[0]:
    if i.item() == 0:
        print("Setosa")
    elif i.item() == 1:
        print("Versicolor")
    else:
        print("Virginica")

Versicolor
Virginica
Versicolor
Virginica
Versicolor
Versicolor
Setosa
Versicolor
Versicolor
Virginica
Setosa
Versicolor
Virginica
Setosa
Versicolor
Versicolor


In [32]:
iris['target_names'] # this line shows the names of outputs and their positions that we used in above code

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')